In [1]:
import warnings
warnings.filterwarnings("ignore")
#Working on plotly
import chart_studio.plotly as py
import cufflinks as cf
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
cf.go_offline()
import pandas as pd
from scipy.stats import kurtosis
from statsmodels.tsa.api import ExponentialSmoothing,SimpleExpSmoothing,Holt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [2]:
df=pd.read_csv(r"C:\Users\2000139\ShurovoData_2018H2\ENV3_1.txt")
df.head()

,Number of points: 8192
0,Sampling rate(Hz): 1562.5
1,Time range: From: 07/01/2018 08:00:12; Until: ...
2,Units: g
3,01.07.2018 08:00:12.7250048\t0.776226
4,01.07.2018 08:00:12.7256448\t0.8002694


In [3]:
df.columns=['ENV3_1']
df = pd.DataFrame(df['ENV3_1'].str.split('\t',2).tolist())

In [4]:
df.dropna(inplace=True)
df[0]=pd.to_datetime(df[0])
df.index=df[0]
df.index=df.index.date
df.drop(columns=[0],inplace=True)
col_name = 'ENV3_1'
df.columns=[col_name]
df.isnull().sum()
df=df.astype({col_name:float,})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1893243 entries, 2018-01-07 to 2018-01-12
Data columns (total 1 columns):
 #   Column  Dtype  
---  ------  -----  
 0   ENV3_1  float64
dtypes: float64(1)
memory usage: 28.9+ MB


In [5]:
df

,ENV3_1
2018-01-07,0.776226
2018-01-07,0.800269
2018-01-07,0.636036
2018-01-07,0.640104
2018-01-07,0.653051
...,...
2018-01-12,1.004822
2018-01-12,1.018878
2018-01-12,1.004082
2018-01-12,1.064005


In [6]:
# outlier_df=df[df['ENV3_1']<2]
# outlier_df
# df.drop(df[df['ENV3_1'] <2].index, inplace= True)

In [7]:
df.index=df.index.astype('str')

In [8]:
kurtosis(df.loc['2018-01-07'],fisher=False)

array([3.23867948])

In [9]:
final_df=df.groupby(df.index).apply(lambda x:kurtosis(x,fisher=False))
#final_df=df.groupby(df.index).agg({'square_values':['mean']})

In [10]:
final_df=pd.DataFrame(final_df)
final_df

,0
2018-01-07,[3.238679477920565]
2018-01-09,[3.299933896122822]
2018-01-10,[3.1579475431749104]
2018-01-12,[2.939850389443223]
2018-02-07,[3.284835848821266]
...,...
2018-11-25,[3.255656811632815]
2018-11-26,[3.195921086195989]
2018-11-27,[3.2647372995132415]
2018-12-07,[3.2514393223894134]


In [11]:
final_df[0]=final_df[0].astype('str')

In [12]:
final_df[0]=final_df[0].apply(lambda x:x[1:-1])
final_df[0]=final_df[0].astype('float')

In [13]:
final_df.columns=['Kurtosis']
final_df

,Kurtosis
2018-01-07,3.238679
2018-01-09,3.299934
2018-01-10,3.157948
2018-01-12,2.939850
2018-02-07,3.284836
...,...
2018-11-25,3.255657
2018-11-26,3.195921
2018-11-27,3.264737
2018-12-07,3.251439


In [14]:
# outlier_df=final_df[final_df['Kurtosis']<0]
# outlier_df

In [15]:
final_df_1=final_df[:]
final_df_1
final_df_1.index=final_df_1.index.astype('str')

final_df_1.index=pd.to_datetime(final_df_1.index)
fit1 = SimpleExpSmoothing(final_df_1['Kurtosis']).fit(smoothing_level=0.2,optimized = False)
final_df_1['Smooth Kurtosis']=list(fit1.fittedvalues)

C:\Users\2000139\AppData\Roaming\Python\Python38\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



In [16]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=final_df_1.index,y=final_df_1['Smooth Kurtosis'],mode='lines',name = 'Smooth Kurtosis',line=dict(color="#ffae42")))
fig.add_trace(go.Scatter(x=final_df_1.index,y=final_df_1.Kurtosis,mode='markers',name = 'Kurtosis',line=dict(color="#5dd55d")))
#fig.add_trace(go.Scatter(x=outlier_df.index,y=outlier_df.RMS,mode='markers',name = 'Non Running',line=dict(color="#ff0000")))
fig.update_layout(hovermode="x")
fig.update_layout(xaxis=dict(showgrid=True),yaxis=dict(showgrid=True),title={'text':'KURTOSIS DATA','xanchor':'center','x':0.5})
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Amplitude')